# Utilisation de la mémoire persistante
Dans cet exercice, nous allons explorer comment intégrer une mémoire persistante dans un agent basé sur un LLM. Contrairement aux systèmes traditionnels qui perdent leur contexte après chaque interaction, un agent avec mémoire peut se souvenir des informations échangées précédemment, ce qui permet de créer des interactions plus naturelles et personnalisées.

## Définition des variables
Les variables sont lues depuis le fichier [.env](../../.env)

In [1]:
from dotenv import dotenv_values
config = dotenv_values("../../.env")

llm_model = config.get('LLM_MODEL')
api_key = config.get('LLM_API_KEY')
mem0_api_key = config.get('MEM0_API_KEY')
# Uncomment for local api call
# api_base = config.get('LLM_API_URL')

## Configuration du llm sur dspy

In [2]:
import dspy

lm = dspy.LM(llm_model, api_key=api_key)
# Uncomment for local api call
#lm = dspy.LM(llm_model, api_base=api_base, track_usage=True, temperature=1.5, max_tokens=1024)

dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)
dspy.configure(lm=lm)

## Créer un client MCP Mem0

Cette section montre comment utiliser le server MCP pour intégrer la mémoire persistante. Nous allons créer des fonctions qui permettent d'ajouter, récupérer et mettre à jour les informations dans la mémoire.

In [3]:
from fastmcp import Client

config = {
    "mcpServers": {
        "Mem0": {
            "command": "uvx",
            "args": ["mem0-mcp-server"],
            "env": {
                "MEM0_API_KEY": mem0_api_key,
                "MEM0_DEFAULT_USER_ID": "default-user-id"
            }
        }
    }
}

mcpClient = Client(config)
tools = []
async with mcpClient:
    # Conversion des MCP tools en DSPy tools
    for tool in await mcpClient.list_tools():
        tools.append(dspy.Tool.from_mcp_tool(mcpClient.session, tool))

## Créer un assistant avec mémoire persistante

Cette section définit comment utiliser la mémoire pour enrichir les interactions. L'assistant peut se souvenir des préférences et informations fournies par l'utilisateur dans ses échanges précédents.

In [4]:
assistant = dspy.ReAct("question -> answer", tools=tools)

## Exécuter l'assistant avec mémoire

In [5]:

import time
from rich import print

results = await assistant.acall(question="Je m'appelle Etienne daho et j'aime les burritos")
print(results.answer)
dspy.inspect_history(1)

time.sleep(60) 

results = await assistant.acall(question="Rappelle moi mon nom et mon plat préféré")
print(results.answer)
dspy.inspect_history(1)

/workspaces/opensource/kata-ai-agent/.venv/lib/python3.13/site-packages/aiohttp/connector.py:963: DeprecationWarning: enable_cleanup_closed ignored because https://github.com/python/cpython/pull/118960 is fixed in Python version sys.version_info(major=3, minor=13, micro=11, releaselevel='final', serial=0)
  super().__init__(


Je m'appelle Etienne Daho et j'aime les burritos.





[2026-02-05T20:00:53.652628]

System message:

Your input fields are:
1. `question` (str): 
2. `trajectory` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.


User message:

[[ ## question ## ]]
Je m'appelle Etienne daho et j'aime les burritos

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
I should store this information about Etienne Daho and his liking for burritos so that it can be referenced later if needed.

[[ ## tool_name_0 ## ]]
add_memory

[[ ## tool_args_0 ## ]]
{"text": "Etienne Daho aime les burritos", "user_id": null, "agent_id": null, "app_id": null, "run_id": null, "metadata": nul

Votre nom est Etienne Daho et votre plat préféré est les burritos.





[2026-02-05T20:02:11.025991]

System message:

Your input fields are:
1. `question` (str): 
2. `trajectory` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.


User message:

[[ ## question ## ]]
Rappelle moi mon nom et mon plat préféré

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
To answer the question, I need to retrieve the user's name and their favorite dish from memories. First, I will search memories related to the user's name and favorite dish to gather this information.

[[ ## tool_name_0 ## ]]
search_memories

[[ ## tool_args_0 ## ]]
{"query": "user's name", "filters": {"AND": [{"user_